In [1]:
from scipy.optimize import differential_evolution
import numpy as np
import json
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"
plt.style.use('seaborn-deep')
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]
from MSC_osteogenesis import *
%config Completer.use_jedi = False
showOriginalModelString = True
study = 'Chen_2018'

In [2]:
# load the samples from the original model
with open('observations.json') as json_file:
    observations = json.load(json_file)
# Set-up hyperparameters
class SETTINGS:
    max_iters = 100
    
def cost_function(calib_params_values):
    # calculate the error for each tag by comparing the results to the original model
    calib_params = {}
    for key,value in zip(free_params.keys(),calib_params_values):
        calib_params[key] = value
    obj = MSC_model(params = calib_params)
    error = obj.run()
    return error

def optimize():
    # Call instance of PSO
    results = differential_evolution(cost_function,bounds=list(free_params.values()),disp=True,maxiter=SETTINGS.max_iters)
    return results
optimize_results = optimize()
print(optimize_results)

differential_evolution step 1: f(x)= 0.76659
differential_evolution step 2: f(x)= 0.76659
differential_evolution step 3: f(x)= 0.656394
differential_evolution step 4: f(x)= 0.656394
differential_evolution step 5: f(x)= 0.656394
differential_evolution step 6: f(x)= 0.382872
differential_evolution step 7: f(x)= 0.175379
differential_evolution step 8: f(x)= 0.175379
differential_evolution step 9: f(x)= 0.175379
differential_evolution step 10: f(x)= 0.139177
differential_evolution step 11: f(x)= 0.139177
differential_evolution step 12: f(x)= 0.125372
differential_evolution step 13: f(x)= 0.125372
differential_evolution step 14: f(x)= 0.125372
differential_evolution step 15: f(x)= 0.117402
differential_evolution step 16: f(x)= 0.117402
differential_evolution step 17: f(x)= 0.110953
differential_evolution step 18: f(x)= 0.0909561
differential_evolution step 19: f(x)= 0.0909561
differential_evolution step 20: f(x)= 0.0909561
differential_evolution step 21: f(x)= 0.0909561
differential_evoluti

In [3]:
# output the inferred params
inferred_params = {}
for key,value in zip(free_params.keys(),optimize_results.x):
    inferred_params[key] = value
# inferred_params['ALP_maturity'] = 100
with open('inferred_params.json','w') as file:
    file.write(json.dumps(inferred_params, indent = 4))

In [4]:
# test it
with open('inferred_params.json') as file:
    inferred_params = json.load(file)
obj = MSC_model(params = inferred_params)
results = obj.simulate_studies()
print(results)

{'Chen_2018': {'ctr': {'ARS': 0.999999998805903}, 'IL10_.01': {'ARS': 1.4767737255960867}, 'IL10_.1': {'ARS': 3.9996787119598123}, 'IL10_100': {'ARS': 0.39978065536097745}}}


In [2]:
class GSs:
    graph_size = [5,5]
    targets = observations[study]['measurement_scheme']
    bar_width = .2
    error_bar_width = .2
    colors = ['lime' , 'violet', 'yellowgreen', 'peru', 'skyblue']
    legend_font_size = 30
    tick_font_size = 30
    title_font_size = 30
    delta = .1 # gap between exp and sim


def sort(results):
    exp_tag_results = {}
    for tag in GSs.targets:
        exp_tag_results[tag] = []
    for tag in GSs.targets:
        for ID in observations[study]['IDs']:
            ID_observations = observations[study][ID]['expectations']
            exp_tag_results[tag].append(ID_observations[tag])
    sim_tag_results = {}
    for tag in GSs.targets:
        sim_tag_results[tag] = []
    for tag in GSs.targets:
        for result in results:
            sim_tag_results[tag].append(result[tag])
    return exp_tag_results,sim_tag_results
def bar_positions():
    for i in range(len(GSs.targets)):
        x_exp =[float(j) + GSs.delta for j in range(len(ID_results))]
        x_sim =[float(j) - GSs.delta for j in range(len(ID_results))]
    return x_exp,x_sim
exp_tag_results,sim_tag_results = sort(results[study])
x_exp,x_sim = bar_positions()
for tag,ii in zip(GSs.targets,range(len(GSs.targets))):
    fig = plt.figure(figsize=(GSs.graph_size[0],GSs.graph_size[1]))
    fig.canvas.draw()
    ax = fig.add_subplot(1, 1, 1)
    ax.bar(x=x_exp,height=exp_tag_results[tag],width = GSs.bar_width, label = 'Exp', facecolor = GSs.colors[ii],
             edgecolor="black", yerr =  0,
             error_kw = dict(capsize= GSs.error_bar_width))
    ax.bar(x=x_sim,height=sim_tag_results[tag],width = GSs.bar_width, label = "Sim", facecolor = GSs.colors[ii+1],
             edgecolor="black", yerr =  0,
             error_kw = dict(capsize= GSs.error_bar_width))
    ax.legend(bbox_to_anchor=(1.4, 1),loc = 'upper right', borderaxespad=0.,prop={ 'family':'Times New Roman','size':GSs.legend_font_size},ncol=1)
#     ax.set_ylim(yrange)
#     labels = [item.get_text() for item in ax.get_xticklabels()]
#     ax.set_xticks(ticks = [int(i) for i in x_labels])
#     ax.set_xticklabels(x_labels_adj)
#     ax.get_yaxis().set_major_formatter(
#         matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x)/1000, ',')))
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontname('Times New Roman')
        label.set_fontsize(GSs.tick_font_size)
    ax.set_ylabel('yaxis_title',fontdict ={'family':'Times New Roman','size':GSs.title_font_size})
    ax.set_xlabel('Days',fontdict ={'family':'Times New Roman','size':GSs.title_font_size})
    
    ax.set_title(tag,fontdict ={'family':'Times New Roman','size':GSs.title_font_size, 'fontweight':'bold'})


NameError: name 'observations' is not defined